# VA Project Template

This template just loads and uses a few of the discussed libraries. Please follow the instruction in Moodle and feel free to remove/update any cells below.

In [ ]:
#disable some annoying warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#plots the figures in place instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np

import altair as alt

import ipywidgets as widgets

# Load Data

In [ ]:
# helper functions for data loading 

def getYearsOfInterest(fromYear, toYear):
    return [str(x) for x in range(fromYear, toYear+1)]

def filterData(valueColumns, metaDataColumns, data):
    missingColumns = list(set(valueColumns) - set(data.columns))
    for c in missingColumns:
        data[c] = None 
    return data[list(set(metaDataColumns) | set(valueColumns))]

def unpivot(data, key_columns, data_column, value_column):
    return pd.melt(data, id_vars=key_columns, var_name=data_column, value_name=value_column)

def loadSingleDataset(path, from_year, to_year, key_columns, data_column, value_column):
    data = pd.read_csv(path) 
    data = filterData(getYearsOfInterest(from_year, to_year), key_columns, data)
    return unpivot(data, key_columns, data_column, value_column)

def mergeDatasets(datasets, keys):
    data = datasets[0]
    
    for i in range(1, len(datasets)):
        data = data.merge(datasets[i], how='outer', left_on=keys, right_on=keys)
        
    return data

In [ ]:
# global report params 
FROM_YEAR = 1900
TO_YEAR   = 2020

In [ ]:
gdp_growth = loadSingleDataset('data/gdp_total_yearly_growth.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'gdp_growth')
gdp_growth.head()


In [ ]:
children_per_woman_total_fertility = loadSingleDataset('data/children_per_woman_total_fertility.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'children_per_woman_total_fertility')
children_per_woman_total_fertility.head()

In [ ]:
co2_emissions_tonnes_per_person = loadSingleDataset('data/co2_emissions_tonnes_per_person.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'co2_emissions_tonnes_per_person')
co2_emissions_tonnes_per_person.head()


In [ ]:
mean_years_in_school_women_percent_men_25_to_34_years = loadSingleDataset('data/mean_years_in_school_women_percent_men_25_to_34_years.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'mean_years_in_school_women_percent_men_25_to_34_years')
mean_years_in_school_women_percent_men_25_to_34_years.head()

In [ ]:
average_age_of_dollar_billionaires_years = loadSingleDataset('data/average_age_of_dollar_billionaires_years.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'average_age_of_dollar_billionaires_years')
average_age_of_dollar_billionaires_years.head()

In [ ]:
food_supply= loadSingleDataset('data/food_supply.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'food_supply')
food_supply.head()

## TODO 
* Add more datasets and then merge them in the following cell  

In [ ]:
hourly_compensation = loadSingleDataset('data/hourly_compensation.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'hourly_compensation')
hourly_compensation.head()

In [ ]:
income_per_person= loadSingleDataset('data/income_per_person.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'income_per_person')
income_per_person.head()

In [ ]:
suicide_per_100000_people = loadSingleDataset('data/suicide_per_100000_people.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'suicide_per_100000_people')
suicide_per_100000_people.head()

In [ ]:
total_number_of_dollar_billionaires = loadSingleDataset('data/total_number_of_dollar_billionaires.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'total_number_of_dollar_billionaires')
total_number_of_dollar_billionaires.head()

In [ ]:
working_hours_per_week = loadSingleDataset('data/working_hours_per_week.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'working_hours_per_week')
working_hours_per_week.head()

## The final merged dataset

* Call mergeDatasets function to form the final dataset
* Augment data with additional attributes (e.g. continent and region data for _'country'_ and decade for _'year'_)

In [ ]:
# merge the datasets in one that contains all the data
data = mergeDatasets([
    gdp_growth, 
    children_per_woman_total_fertility,
    co2_emissions_tonnes_per_person,
    mean_years_in_school_women_percent_men_25_to_34_years,
    average_age_of_dollar_billionaires_years,
    food_supply,
    hourly_compensation,
    income_per_person,
    suicide_per_100000_people,
    total_number_of_dollar_billionaires,
    working_hours_per_week
], ['country', 'year'])

data.sort_values(by=['country', 'year'], inplace=True, ignore_index=True)


countries = pd.read_csv('data/countryContinent.csv')

data = data.merge(countries, how='left', left_on=['country'], right_on=['country'])
data = data.convert_dtypes()

#add 'decade' computed column 
data['decade'] = data['year'].str.slice(0, 3)  + '0'

#check for missing countries (they have to be corrected in countryContinent.csv)
missing_countries = data[data["region_code"].isnull()]['country'].unique()

if (len(missing_countries) == 0):
    print("Country mapping is OK")
else:
    print(missing_countries)
    
data.to_csv('data/data.csv')

In [ ]:
# basic statistics of the loaded data 
print(data.count())
data.head(50)

## Helper variables for different set of columns in the dataset

In [ ]:
# change all _ with line breaks - in this case it's easier to display column names in the plots
mapping = {}
for col in data:
    mapping[col] = col.replace('_', "\n")
    
data = data.rename(columns=mapping)

key_columns = ['country', 'year']

measure_columns = [
        "gdp\ngrowth",
        "children\nper\nwoman\ntotal\nfertility",
        "co2\nemissions\ntonnes\nper\nperson",
        "mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears"
        #"average\nage\nof\ndollar\nbillionaires\nyears",
        #'food_supply',
        #'hourly_compensation',
        #'income_per_person',
        #'suicide_per_100000_people',
        #'total_number_of_dollar_billionaires',
        #'working_hours_per_week'
    ]

all_columns = key_columns + measure_columns

# Show Data

## Step 1 - Data Completeness
 > In the data quality framework, data completeness refers to the degree to which all data in a data set is available. A measure of data completeness is the percentage of missing data entries [[1]](https://dataladder.com/missing-data-and-data-completeness/)

In [ ]:
t1 = pd.melt(data[all_columns], id_vars=['country', 'year'], var_name=['measure'], value_name='val')
t1['Countries Count'] = t1['val'].isnull()

t1 = t1.groupby(['year', 'measure'])['Countries Count'].sum().reset_index()

alt.Chart(t1).mark_rect().encode(
    x='year:O',
    y='measure:O',
    color='Countries Count:Q'
).properties(
    width=800,
    height=300,
    title='Data Completeness'
)


## Step 2 - Simple Plots

### 2.1. Interactive Scatter Plot Matrix.

The goal of this visualization is to explore the correlations between data columns.
The data is filtered by country.

In [ ]:
@widgets.interact(country=data['country'].unique())
def basic_plot(country):
    filtered_df = data.loc[(data['country'] == country)]
    #print(filtered_df[measure_columns].describe())
    pd.plotting.scatter_matrix(filtered_df[measure_columns], alpha=0.5, figsize=(9,9))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

import ipywidgets as widgets

from ipywidgets import interact, interact_manual

In [ ]:
basis = pd.read_csv('data/data.csv')
basis.head(50)

### DESCRIPTIVE STATISTICS.

Analyzing our dataset using descriptive statistics on the level of individual attributes.
This includes simple plots of distributions and statistics.

In [ ]:
alt.data_transformers.disable_max_rows()
billionaires = alt.Chart(basis).mark_bar().encode(
    alt.X('average_age_of_dollar_billionaires_years', bin= True, scale=alt.Scale(zero=False), title='average age'),
    alt.Y('total_number_of_dollar_billionaires', title='count')
).properties(
    title='Count of average age of dollar billionaires'
)

billionaires

As expected the majority of dollar billionaires are aged 60-70, but we can also see some people in the 40-50 category and even a minority in the 20-30, 30-40 respectively field.
Would be interesting to see which countries they are from.

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1, 2, 1)
sns.boxenplot(data = basis.loc[basis["year"]==1960], 
              x="continent",y="children_per_woman_total_fertility")
#sns.swarmplot(data  = basis.loc[basis["year"]==1960], 
              #x="continent",y="children_per_woman_total_fertility",
              #color="0000")
plt.title("fertility 1960")

plt.subplot(1, 2, 2)
sns.boxenplot(data = basis.loc[basis["year"]==2020], 
              x="continent",y="children_per_woman_total_fertility")
#sns.swarmplot(data  = basis.loc[basis["year"]==2020], 
              #x="continent",y="children_per_woman_total_fertility",
              #color="0000")
plt.title("fertility 2020")

plt.show()



The graphs reveal that the amount of babies per woman has dropped in every continent during the last 60 years.
More evident in Asia, America and Ocenania. While Europe's change in the past 60 years wasn't as significant.
Even in Africa the count of babies per woman has dropped, but still remains higher than the other continents.

In [ ]:
#not sure why it's displaying the ranking, but the bars keep showing the max instead of mean.
#and suddenly also includes countries with no value.
alt.Chart(basis).mark_bar().encode(
    x=alt.X('suicide_per_100000_people:Q'),
    y=alt.Y('country', sort=alt.EncodingSortField(field="suicide_per_100000_people", op="mean", order='descending'))
)


Suicide rate of the top 5 countries include regions except for Hungary that formerly belong to the UdSSR.

In [ ]:
#do not why the chart is just showing 5 countries although I've set it to top 20. ^.^'
alt.Chart(basis).mark_bar().encode(
    x='suicide_per_100000_people:Q',
    y=alt.Y('country:N',
        sort=alt.EncodingSortField(field='suicide_per_100000_people', op='max', order='descending'))
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'suicide_per_100000_people', 'order': 'descending'}]
).transform_filter('datum.rank <= 20')

### CORRELATIONS.

Analyzing ourdataset by looking at correlations between attributes (dimensions) and coming up with an interpretation why in which way specific attributes are correlated. 

In [ ]:
corr = basis[['income_per_person','working_hours_per_week','hourly_compensation','food_supply',
              'suicide_per_100000_people','total_number_of_dollar_billionaires',
              'average_age_of_dollar_billionaires_years',
              'children_per_woman_total_fertility','co2_emissions_tonnes_per_person',
              'mean_years_in_school_women_percent_men_25_to_34_years','gdp_growth']].corr()
f, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)



Contrary to the hypothesis that income per person and working hours would have an impact on suicide rate the heatmap shows no or even a negative correlation. Meaning money is not the sole reason why people decide to end their lives.
Income per person on the other hand is highly correlated to calories intake and co2 emission per person which is comprehensible given the fact that with higher income there is more money to spend for consumation. 
Interestingly calories intake is also connected to the mean years in school which can be explained by implicit relationship between calories intake and income per person. 
The average age of dollar billionaires is highly correlated to the hourly compensation which makes sense...

In [ ]:
colors = sns.color_palette()

@interact(year=(basis.year.min(), basis.year.max()))
def plot(year):
    basisyear = basis[basis.year == year]
    for (name, group),color in zip(basisyear.groupby('continent'),colors):
        plt.scatter(x=(group['income_per_person']),y=group['food_supply'],
                    label=name, c=[color])#s=(group['pop']/pop_max)*400)
    plt.title('income per person vs calories intake')
    plt.xlabel('income per person')
    plt.ylabel('calories intake')
    plt.legend()

In [ ]:
@widgets.interact(year = basis['year'])
def show_plot(year):
    basisyear = basis[basis.year == year]
    return alt.Chart(basisyear).mark_circle().encode(
    x=alt.X('income_per_person'),
    y='food_supply',
    #y=alt.Y('lifeExp', scale=alt.Scale(zero=False)),
    color='continent',
    #size='pop',
    tooltip=('country','income_per_person')
)


Interestingly a lot of Asian countries that are not mentioned a lot in the media 
have high income per person in the early 60's and keep the status throughout 
the 2000's (Brunei, Kuwait, Saudi Arabia). While keeping the calories intake as 
on a stable level. African countries increase their income per person as well but
not as much as their food supply.


### CLUSTERING.

Clustering similar items and show the clustering results.
1. User can interactively select the cluster algorithms and/or its parameters.
2. User can select one/more cluster/s from the resulting visualization.
3. User can see the selected data with its cluster affiliation in a second interactive visualization (simple overview+detail visualization setup)



In [ ]:
brush = alt.selection_interval()
selection = alt.selection_multi(fields=['continent:N'])


chart = alt.Chart(basis).mark_circle().encode(
    x='working_hours_per_week:O', y= 'hourly_compensation:Q',
    tooltip=['country','hourly_compensation'],
    size='income_per_person',
    color=alt.condition(brush|selection, alt.Color('continent:O', 
                                                   scale=alt.Scale(scheme='category10')),
        alt.value('lightgray'))
).add_selection(
    brush, selection
).properties(
    width=600,
    height=300,
)
chart
